Import the libraries

Utility functions, to store the temporary images and to convert matrix shape.

## Dataset

## Model

In [ ]:
# Creating a PyTorch class
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(8, 16,
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(16, 1,
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

## Training/Loading

In [ ]:
if exists(model_file):
    model.load_state_dict(torch.load(model_file))
    model.eval()
else:
    losses = []
    for epoch in range(num_epochs):
        for data in loader:
            image, label = data

            # Output of Autoencoder
            image = image.to(device)

            _, reconstructed = model(image)
            # Calculating the loss function
            loss = loss_function(reconstructed, image)

            # The gradients are set to zero,
            # the the gradient is computed and stored.
            # .step() performs parameter update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Storing the losses in a list for plotting
            losses.append(loss.cpu().detach().numpy())
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data))
        if epoch % 10 == 0:
            pic = to_img(reconstructed.cpu().data)
            save_image(pic, './mlp_img/image_{}.png'.format(epoch))

    torch.save(model.state_dict(), model_file)
    model.eval()
    # Defining the Plot Style
    plt.style.use('fivethirtyeight')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')

    # Plotting the last 100 values
    plt.plot(losses)

In [ ]:
import random
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid

model.eval()
model.to("cpu")
fig = plt.figure(figsize=(20., 5.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(2, 10),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )

for i in range(10):
    image, _ = dataset.__getitem__(random.randint(1, 37000))

    _, reconstructed = model(image)
    item = reconstructed.reshape(-1, 28, 28)
    item = item.detach().numpy()

    # Iterating over the grid returns the Axes.
    grid[i].imshow(image[0])
    grid[i].grid(False)
    grid[10 + i].imshow(item[0])
    grid[10 + i].grid(False)

plt.savefig("cae_nums.png", transparent=True)
plt.show()